In [48]:
import pandas as pd
from pandas import plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlib
import plotly.plotly as py
import pylab 
import scipy.stats as stats
import statsmodels.api as sm
import seaborn as sns
import math
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder

disease_dat = pd.read_csv('thyroid_desiease.txt')

### Первый взгляд на данные.
Заменим все вопросы в данных на отсутствие значения.

In [49]:
disease_dat=disease_dat.replace('?',np.nan);
disease_dat.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,NaN,SVHC,negative
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,NaN,f,NaN,f,NaN,other,negative
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,NaN,other,negative
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,NaN,f,NaN,f,NaN,other,negative
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,NaN,SVI,negative


Приведем к типу float, значения, которые изначально являлись таковыми, но в процессе загрузки данных стали строковыми.

In [50]:
disease_dat[['age','TSH', 'T3', 'TT4','T4U','FTI']] = disease_dat[['age','TSH', 'T3', 'TT4','T4U','FTI']].apply(pd.to_numeric)

In [51]:
ProfileReport(disease_dat)

Number of variables,30
Number of observations,3772
Total Missing (%),5.4%
Total size in memory,884.1 KiB
Average record size in memory,240.0 B
Numeric,6
Categorical,22
Boolean,0
Date,0
Text (Unique),0
Rejected,2


Среди данных присутствует признак(TBG), для которого нет ни единного значения, его мы исключим из рассмотрения

In [52]:
disease_dat = disease_dat.dropna(axis = 1, thresh=1000)

Заменим значения категориальных признаков на числовые:

f -> 0,  t -> 1,  F -> 0,  M -> 1

In [57]:
dis=pd.get_dummies(disease_dat)
dis.head().T

,0,1,2,3,4
age,41.00,23.0,46.00,70.00,70.00
TSH,1.30,4.1,0.98,0.16,0.72
T3,2.50,2.0,NaN,1.90,1.20
TT4,125.00,102.0,109.00,175.00,61.00
T4U,1.14,NaN,0.91,NaN,0.87
FTI,109.00,NaN,120.00,NaN,70.00
sex_F,1.00,1.0,0.00,1.00,1.00
sex_M,0.00,0.0,1.00,0.00,0.00
on_thyroxine_f,1.00,1.0,1.00,0.00,1.00
on_thyroxine_t,0.00,0.0,0.00,1.00,0.00


Столбики, помеченные как measured, указывают лишь на то, измерены ли соответствующие признаки или нет. Больше они не несут никакой информации, поэтому их следуюет исключить.

В данных указано, что некоторые значения не измерялись, возможно на то не было причин (или наоборот были очень веские причины не измерять), поэтому заменим неизмеренные данные на 0.

In [ ]:
disease = disease_dat.drop(['TSH_measured','T3_measured','TT4_measured','FTI_measured','T4U_measured','TBG_measured'], axis=1)

In [ ]:
disease=disease.fillna(0)

In [ ]:
sns_plot = sns.pairplot(disease[['age','TSH', 'T3', 'TT4','T4U','FTI']])

В данных присутствует возраст равный 455, я полагаю, что человек забивающий данные (или кто бы это ни был), просто два раза нажал на 5. Поэтому сделаю замену данного значения на 45.

In [ ]:
disease.loc[disease.age > 400, 'age'] = 45

## kNN and Tree
### Простой случай
Научимся сначала определять есть ли Гипотериоз или нет, потому что наблюдается сильный дисбаланс в данных лучше сначала опеделить среди большого массива людей с болезнью, а потом уже среди больных определить какое именно у них заболевание, с помощью методов ближайщих соседей и дерева. Затем возьмем данную модель и будем сравнивать с ней все последующие.

In [ ]:
referral_source = disease['referral_source']
Class = disease['Class']
disease_num = disease.drop(['referral_source','Class'], axis=1)
Class.replace('negative',0,inplace=True);
Class.replace('compensated_hypothyroid',1,inplace=True);
Class.replace('primary_hypothyroid',1,inplace=True);
Class.replace('secondary_hypothyroid',1,inplace=True);

In [ ]:
disease_num.info()

In [40]:
def show_APR(y, pred):
    d = {"Metrix":np.array([accuracy_score(y, pred),precision_score(y, pred),recall_score(y, pred)])}
    df = pd.DataFrame(d, index =["Accuracy","Precision","Recall"])
    display(df)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(disease_num.values, Class, test_size=0.3,
random_state=17, stratify=Class)

tree = DecisionTreeClassifier(max_depth=5, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)

tree.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [42]:
tree_pred = tree.predict(X_test)
show_APR(y_test, tree_pred)

,Metrix
Accuracy,0.992933
Precision,0.915789
Recall,1.000000


In [43]:
knn_pred = knn.predict(X_test)
show_APR(y_test, knn_pred)

,Metrix
Accuracy,0.953180
Precision,0.904762
Recall,0.436782


Простая модель дерева показала очень хорошие метрики качества, что нельзя сказать об kNN. Попробуем улучшить метрики обоих алгаритмов, перебором максимальной глубины и количеством используемых фич в дереве, используя GridSearchCV. А в kNN с помощью него поизменять количество соседей.

### Чуть более сложная модель
Глубина будет изменяться от 1 до 11, количество параметров от 2 до 21.

В kNN число ближайших соседей проверим от 1 до 20, так же попробуем запустить разные вариации данного алгоритма.

Так же проведена 10 кратная кроссвалидация.

In [44]:
tree_params = {'max_depth': range(1,11),
               'max_features': range(2,21)}
tree_grid = GridSearchCV(tree, tree_params,cv=10, n_jobs=-1,verbose=True)
tree_grid.fit(X_train, y_train)

Fitting 10 folds for each of 190 candidates, totalling 1900 fits


[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done 1900 out of 1900 | elapsed:   43.2s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': range(2, 21), 'max_depth': range(1, 11)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [45]:
display(tree_grid.best_params_)
display(tree_grid.best_score_)
show_APR(y_test, tree_grid.predict(X_test))

{'max_depth': 4, 'max_features': 18}

0.99696969696969695

,Metrix
Accuracy,0.995583
Precision,0.945652
Recall,1.000000


Результаты показали, что оптимальная глубина 4 уровням, а оптимальное число параметров равно 15 (scoring ставил Recall результаты по данной метрике были такими же, но остальные метрики ухудшились). Результаты метрик при таких параметрах показывают очень хорошие результаты. Хотя Recall не достигла 1. В идеале нужно всех больных классифицировать как больных, даже если это немного уменьшит Precision, потому что штраф за ошибку первого рода больше, чем за ошибку второго рода. Accuracy на кроссвалиации равно 99,73%, на отложенной выборке 99,73%.

In [46]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(1, 20)}
knn_grid = GridSearchCV(knn_pipe, knn_params,cv=10, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)

Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed: 18.3min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'knn__n_neighbors': range(1, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [47]:
knn_grid.best_params_
knn_grid.best_score_
show_APR(y_test, knn_grid.predict(X_test))

,Metrix
Accuracy,0.952297
Precision,0.971429
Recall,0.390805


Для kNN результаты хуже, чем для tree. Попробуем разные алгоритмы, также поставим, чтобы улучшаемая метрика была recall, так как нам важно, чтобы все больные были правильно опознаны.

In [35]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1,algorithm='kd_tree'))])
knn_params = {'knn__n_neighbors': range(1, 20)}
knn_grid = GridSearchCV(knn_pipe, knn_params,scoring='recall',cv=10, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)
knn_grid.best_params_
knn_grid.best_score_
show_APR(y_test, knn_grid.predict(X_test))

Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:   38.0s finished


,Metrix
Accuracy,0.943463
Precision,0.562500
Recall,0.500000


In [36]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1,algorithm='brute'))])
knn_params = {'knn__n_neighbors': range(1, 20)}
knn_grid = GridSearchCV(knn_pipe, knn_params,scoring='recall',cv=10, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)
knn_grid.best_params_
knn_grid.best_score_
show_APR(y_test, knn_grid.predict(X_test))

Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:   36.8s finished


,Metrix
Accuracy,0.943463
Precision,0.562500
Recall,0.500000


Простым брутфорсом плохой результат получен быстре, но значение recall очень маленькое в сравнении с лесом.))